In [ ]:
## Tested using the following loaded modules
## module use /g/data/access/ngm/modules
## module load analysis3/21.10
import os, sys
workdir = "/g/data/xv83/users/bxn599/ACS/evaluation/"
os.chdir(workdir)
sys.path.append("./lib")
import glob
import xarray as xr
import numpy as np
import geopandas as gp
import datetime as dt
import lib_standards
import lib_spatial
import spatial_selection
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
importlib.reload(lib_standards)
import warnings
warnings.filterwarnings('ignore')
import csv
import barpa_drs_interface as ccam_drs_interface
import re

font = {'size'   : 12}
mpl.rc('font', **font)

In [ ]:
GCMS=['EC-Earth-Consortium-EC-Earth3','NCC-NorESM2-MM']
SCENARIOS=['ssp126','ssp370']
PERIOD='ACS_HISTORICAL'
LATEPERIODS = {'ACS_2020': '2020-2039', 'ACS_2040': '2040-2069', 'ACS_2060': '2060-2079', 'ACS_2080': '2080-2099'}
freq='mon'
VARS=['pr','tasmean']
REGION='Australia'
SEASONS=['all', 'DJF', 'MAM', 'JJA', 'SON']

In [ ]:
header = ['Season', 'Variable', 'Project', 'Domain', 'Downscaler', 'Region', 'Model', 'Experiment', 'Variant', 'Hist-Epoch', 'Proj-Epoch', 'Change-Type', 'Value']
f = open('/g/data/xv83/users/bxn599/ACS/evaluation/test_raw_barpa.csv','w')
writer = csv.writer(f)
writer.writerow(header)

In [ ]:
for var in VARS:
    for SEASON in SEASONS:
        if SEASON == 'all':
            season = 'annual'
        else:
            season = SEASON
        for gcm in GCMS:
            model = gcm
            experiment = ccam_drs_interface._GCM_ENS[gcm]
            dhist = ccam_drs_interface.get_barpa(gcm, 'historical', 'mon', var, [dt.datetime(1995,1,1), dt.datetime(2014,12,31,23)], as_iris=False, lonrange=[112.0,156.2], latrange=[-44.5,-10.0],verbose=False)
            dhist = lib_standards.standardise_data(dhist, region='Australia', period=PERIOD, season=SEASON, compute=True)
            print(dhist)
            d0 = dhist[var].mean(dim='time')
            d0 = lib_spatial.apply_region_mask(d0, REGION)
            for scenario in SCENARIOS:
                dfut = ccam_drs_interface.get_barpa(gcm, scenario, 'mon', var, [dt.datetime(2015,1,1), dt.datetime(2099,12,31,23)], as_iris=False, lonrange=[112.0,156.2], latrange=[-44.5,-10.0],verbose=False)
                for LATEPERIOD in LATEPERIODS:
                    print(LATEPERIOD)
                    print(model)
                    print(experiment)
                    print(LATEPERIOD)
                    futperiod=LATEPERIODS[LATEPERIOD]
                    print(futperiod)
                    print(season)
                    print(REGION)
                    print(scenario)
                    dfuts = lib_standards.standardise_data(dfut, region='Australia', period=LATEPERIOD, season=SEASON, compute=True)
                    print(dfuts)
                    d1 = dfuts[var].mean(dim='time')
                    d1 = lib_spatial.apply_region_mask(d1, REGION)
                    d2 = (d1-d0)/d0*100 # percentage change
                    unweighted = d2.mean(("lon","lat"))

                    d1_mean = dfuts[var].mean()
                    d0_mean = dhist[var].mean()
                    d2_mean = (d1_mean-d0_mean)/d0_mean*100
                    
                    weights = np.cos(np.deg2rad(d2.lat))
                    weights.name = "weights"
                    area_avg = d2.weighted(weights)
                    area_avg = d2.mean(("lon", "lat"))
                    print(area_avg.values)
                    print(unweighted.values)
                    print(d2_mean.values)
                    data = [season, var, 'BARPA-R', 'AUS-15', 'n/a', REGION, model, scenario, experiment, '1995-2014', futperiod, 'percent', area_avg.values]
                    writer.writerow(data)

In [ ]:
f.close()